In [2]:
import pickle
import sys
import os
import datetime

import sklearn
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.datasets import dump_svmlight_file
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

from subprocess import call

In [3]:
#Import data from the processing of data in the journal_training_word2vec notebook
#df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/article_replication/df.p', 'rb'))
df = pickle.load(open('/mnt/mypartition/Desktop2/dfJournals_trans_categories.p', 'rb'))

In [7]:
print df.shape
print df.columns.values

(97049, 5)
['label' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']


In [6]:
#for the journals VM dataframe, change the label of the 'sentiment' column to 'label'
df.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']

In [8]:
df[0:10]

,label,title,abstract,qualifier_terms,descriptor_terms
0,T0,Quantification of the relative contributions o...,Amyotrophic lateral sclerosis (ALS) and fronto...,,
1,T0,Ubiquitin Specific Protease 36 (USP36) control...,Ubiquitination of the TrkA neurotrophin recept...,,
2,T0,Functional Diversity of Cytotoxic tRNase/immun...,Contact-dependent growth inhibition (CDI) is a...,,
3,T0,Identification of a distinct substrate binding...,The type III secretion system (T3SS) effector ...,,
4,T0,Role of chondroitin sulfate (CS) modification ...,Protein tyrosine phosphatase receptor type Z (...,,
5,T0,Chemically programmed bispecific antibodies in...,Chemically programmed bispecific antibodies (b...,,
6,T0,Non-mutagenic Suppression of Enterocyte Ferrop...,Iron transfer across the basolateral membrane ...,,
7,T0,Structural plasticity of cholesteryl ester tra...,Cholesteryl ester transfer protein (CETP) medi...,,
8,T0,Identification of a Membrane-Bound Prepore Spe...,Pore-forming toxins (PFT) are cytolytic protei...,,
9,T0,EspR-dependent ESAT-6 secretion of Mycobacteri...,Attenuation of M. bovis BCG strain is related ...,,


In [65]:
#on local:
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [ ]:
a = datetime.datetime.now().replace(microsecond=0)



b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

In [77]:
def stratified_5_fold_cv(df, mesh_terms, title, label):

    df['boolean_label'] = 1
    for row in df.iterrows():
        if row[1]['label'] != label:
            df.set_value(row[0], 'boolean_label', 0)
        
    #5-fold stratified cross validation
    #because no validation set, 4/5 of values go to train and 1/5 to test
    y = df['boolean_label'].values
    skf = StratifiedKFold(y, n_folds=5, shuffle=True)

    j = 1
    avg = []
    for train_index, test_index in skf:
        print("ROUND", j)
        j = j + 1
        #use the indexes to subset the df pandas dataframe
        train1, test1 = df.iloc[train_index], df.iloc[test_index]

        print "Cleaning and parsing the training set abstracts...\n"
        # Initialize an empty list to hold the clean reviews
        clean_train_reviews = []

        # Loop over each review; create an index i that goes from 0 to the length
        # of the pandas df column
        for i in range(train1["abstract"].size):
            # Call our function for each one, and add the result to the list of
            # clean reviews
            words = KaggleWord2VecUtility.review_to_words( train1["title"].iloc[i])
            words = words + KaggleWord2VecUtility.review_to_words( train1["descriptor_terms"].iloc[i])
            words = words + KaggleWord2VecUtility.review_to_words( train1["qualifier_terms"].iloc[i])
            words = words + KaggleWord2VecUtility.review_to_words( train1["abstract"].iloc[i])
            clean_train_reviews.append(words)

        #with tfidf word weighting
        #uses L2 norm by default
        print "Creating the bag of words...\n"

        # Initialize the "TfidfVectorizer" object, which is scikit-learn's
        # bag of words tool.  
        vectorizer = TfidfVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None)
                                #max_features = 5000

        # fit_transform() does two functions: First, it fits the model
        # and learns the vocabulary; second, it transforms our training data
        # into feature vectors. The input to fit_transform should be a list of 
        # strings.
        train_data_features = vectorizer.fit_transform(clean_train_reviews)

        # Numpy arrays are easy to work with, so convert the result to an 
        # array
        train_data_features = train_data_features.toarray()

        #Moving on to test set...
        # Create an empty list and append the clean reviews one by one
        clean_test_reviews = [] 

        print "Cleaning and parsing the test set abstracts...\n"
        for i in range(test1["abstract"].size):    
            words = KaggleWord2VecUtility.review_to_words( test1["title"].iloc[i])
            words = words + KaggleWord2VecUtility.review_to_words( test1["descriptor_terms"].iloc[i])
            words = words + KaggleWord2VecUtility.review_to_words( test1["qualifier_terms"].iloc[i])
            words = words + KaggleWord2VecUtility.review_to_words( test1["abstract"].iloc[i])
            clean_test_reviews.append(words)

        print "Adding tfidf weights..."
        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()

        #------------------------------------------------------------------------------------------------------------    
        print "Training the random forest..."

        X = train_data_features
        y = train1.boolean_label

        dump_svmlight_file(X,y,'svmlight.dat',zero_based=True,multilabel=False)

        X_test = test_data_features
        y_test = test1.boolean_label

        dump_svmlight_file(X_test,y_test,'svmlight_test.dat',zero_based=True,multilabel=False)

        call(["/home/sarahwie/Downloads/fest/festlearn", "-c 3", 
          "./svmlight.dat", "modelA"])

        print "Using the random forest to make sentiment label predictions..."\

        #get predictions for test set
        call(["/home/sarahwie/Downloads/fest/festclassify", 
          "./svmlight_test.dat", "modelA", "./probs"])

        print "Scoring the test set"
        probs = pd.read_table('./probs', header=None)
        probs.columns = ['probs']

        predictions = np.ones((probs.shape[0]))
        predictions[np.where(probs.iloc[:,0] < 0.5)] = 0 # The first column is the negative model

        score = roc_auc_score(test1["boolean_label"], predictions)
        print score

        #append to average
        avg.append(score)


    print("Average score of 5 rotations:", sum(avg)/float(len(avg)))
    


('ROUND', 1)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Cleaning and parsing the test set abstracts...

Adding tfidf weights...
Training the random forest...
Using the random forest to make sentiment label predictions...
Scoring the test set
0.607142857143
('ROUND', 2)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Cleaning and parsing the test set abstracts...

Adding tfidf weights...
Training the random forest...
Using the random forest to make sentiment label predictions...
Scoring the test set
0.678571428571
('ROUND', 3)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Cleaning and parsing the test set abstracts...

Adding tfidf weights...
Training the random forest...
Using the random forest to make sentiment label predictions...
Scoring the test set
0.730769230769
('ROUND', 4)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Cleaning and pars

In [8]:
#get training and testing set just for our purposes
fracTrain = 0.5
nSamples = df.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = df.ix[order[:splitIndex],:]
test1 = df.ix[order[splitIndex:],:]
print nSamples
print train1.shape[0]
print test1.shape[0]

354
177
177


In [9]:
print "Cleaning and parsing the training set abstracts...\n"
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the pandas df column
for i in range(train1["abstract"].size):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    words = KaggleWord2VecUtility.review_to_words( train1["title"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( train1["descriptor_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( train1["qualifier_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( train1["abstract"].iloc[i])
    clean_train_reviews.append(words)

#with tfidf word weighting
#uses L2 norm by default
print "Creating the bag of words...\n"

# Initialize the "TfidfVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = TfidfVectorizer(analyzer = "word",   \
                         tokenizer = None,    \
                         preprocessor = None, \
                         stop_words = None)
                        #max_features = 5000

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Cleaning and parsing the training set abstracts...



/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Creating the bag of words...



In [40]:
#Moving on to test set...
# Create an empty list and append the clean reviews one by one
clean_test_reviews = [] 

print "Cleaning and parsing the test set abstracts...\n"
for i in range(test1["abstract"].size):    
    words = KaggleWord2VecUtility.review_to_words( test1["title"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( test1["descriptor_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( test1["qualifier_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( test1["abstract"].iloc[i])
    clean_test_reviews.append(words)

print "Adding tfidf weights..."
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

Cleaning and parsing the test set abstracts...

Adding tfidf weights...


In [10]:
print train_data_features
print train_data_features.shape

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(177, 6564)


In [41]:
print test_data_features
print test_data_features.shape

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(177, 6564)


In [11]:
os.getcwd()

'/home/sarahwie/Documents/pubmed-nlp-research/article_replication'

In [17]:
#X = df[np.setdiff1d(df.columns,['pmid','label', 'boolean_label'])]
#y = df.boolean_label

#print df[0:5]
#print X[0:5]

X = train_data_features
y = train1.boolean_label

dump_svmlight_file(X,y,'svmlight.dat',zero_based=True,multilabel=False)

In [42]:
X_test = test_data_features
y_test = test1.boolean_label

dump_svmlight_file(X_test,y_test,'svmlight_test.dat',zero_based=True,multilabel=False)

This file is now ready to be input to a ML algorithm (e.g. random forest FEST).

### Now: implement FEST C files here in python.

In [20]:
os.chdir('/home/sarahwie/Documents/pubmed-nlp-research/article_replication/')

In [21]:
#didn't work really
X_train, y_train = load_svmlight_file('svmlight.dat')

In [14]:
print X_train.shape
print y_train.shape
#labels
print y_train
print type(X_train)
print type(y_train)

(177, 6564)
(177,)
[ 1.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.
  1.  1.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  1.  1.  0.  0.
  1.  1.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.
  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.]
<class 'scipy.sparse.csr.csr_matrix'>
<type 'numpy.ndarray'>


### Now onto FEST implementation

In [44]:
os.getcwd()

'/home/sarahwie/Documents/pubmed-nlp-research/article_replication'

In [43]:
from subprocess import check_call
call(["/home/sarahwie/Downloads/fest/festlearn", "-c 3", 
      "./svmlight.dat", "modelA"])

0

In [53]:
#get predictions for test set
call(["/home/sarahwie/Downloads/fest/festclassify", 
      "./svmlight_test.dat", "modelA", "./probs"])

0

In [56]:
probs = pd.read_table('./probs', header=None)

In [57]:
probs.columns = ['probs']
print probs

        probs
0    0.205000
1    0.368215
2    0.050000
3    0.105000
4    0.040000
5    0.110000
6    0.399999
7    0.434999
8    0.125000
9    0.085000
10   0.215000
11   0.085001
12   0.240000
13   0.150000
14   0.534999
15   0.069983
16   0.219999
17   0.120000
18   0.147500
19   0.110000
20   0.130000
21   0.140000
22   0.305000
23   0.175000
24   0.115000
25   0.298333
26   0.279999
27   0.439999
28   0.430000
29   0.093216
..        ...
147  0.315000
148  0.205000
149  0.140000
150  0.185000
151  0.269999
152  0.050000
153  0.160000
154  0.275833
155  0.439999
156  0.105000
157  0.120000
158  0.230000
159  0.213216
160  0.505000
161  0.230000
162  0.315002
163  0.055000
164  0.143636
165  0.090000
166  0.135001
167  0.474999
168  0.360000
169  0.200000
170  0.185000
171  0.130000
172  0.299999
173  0.370000
174  0.145000
175  0.190000
176  0.160000

[177 rows x 1 columns]


In [58]:
predictions = np.ones((probs.shape[0]))
print np.shape(predictions)
predictions[np.where(probs.iloc[:,0] < 0.5)] = 0 # The first column is the negative model
print predictions

(177,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [59]:
print predictions.shape
print test1["boolean_label"].shape

(177,)
(177,)


In [63]:
#get AUC score
score = roc_auc_score(test1["boolean_label"], predictions)
#core = np.size(np.where(predictions == test1["boolean_label"]))*1./np.size(predictions)
print score

0.533703590992


In [5]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences